In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import onn

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo

In [4]:
def gen_pattern_replace_and_matcher_for_ffm(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher
  def _match(match,ori,pat):
    return True 
  class PatternClass(torch.nn.Module):
    def __init__(self,num_fields):
      super().__init__()
      self.num_fields = num_fields
      self.embeddings = torch.nn.ModuleList([
              torch.nn.Embedding(1, 1) for _ in range(self.num_fields-1)
          ])
    def field_aware_interaction(self,field_aware_emb_list):
      interaction = []
      for i in range(self.num_fields - 1):
          for j in range(i + 1, self.num_fields):
              v_ij = field_aware_emb_list[j - 1][:, i, :]
              v_ji = field_aware_emb_list[i][:, j, :]
              dot = torch.sum(v_ij * v_ji, dim=1, keepdim=True)
              interaction.append(dot)
      return torch.cat(interaction, dim=1)  
    def forward(self,x):
      field_aware_emb_list = [self.embeddings[i](x) for i in range(self.num_fields-1)]
      return self.field_aware_interaction(field_aware_emb_list)
  pattern = PatternClass(num_field)  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  embed_names = [f'embeddings_{i}' for i in range(num_field-1)]
  embed_node_list = [node_map[pattern_env[name]] for name in embed_names]
  embed_node_module_list = [utils.get_target_mod(traced,embed_node.target) for embed_node in embed_node_list]
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embeddings = torch.nn.ModuleList(embed_node_module_list)
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      self.num_sufix = self.num_fields - self.num_prefix
    def forward(self,x):
      redundancy_part = x[redundancy_part_slice] 
      non_redundancy_part = x[non_redundancy_part_slice] 
      redundancy_ffm_embed = [self.embeddings[i](redundancy_part) for i in range(self.num_fields-1)]
      non_redundancy_ffm_embed = [self.embeddings[i](non_redundancy_part) for i in range(self.num_fields-1)]
      redundancy_interaction = []
      for i in range(self.num_prefix - 1):
          for j in range(i + 1, self.num_prefix):
              v_ij = redundancy_ffm_embed[j - 1][ i, :]
              v_ji = redundancy_ffm_embed[i][ j, :]
              dot = torch.sum(v_ij * v_ji, dim=-1, keepdim=True)
              redundancy_interaction.append(dot)
              
      non_redundancy_interaction = []
      for i in range(self.num_sufix - 1):
          for j in range(i + 1, self.num_sufix):
              v_ij = non_redundancy_ffm_embed[j - 1+self.num_prefix][:, i, :]
              v_ji = non_redundancy_ffm_embed[i+self.num_prefix][:, j, :]
              dot = torch.sum(v_ij * v_ji, dim=-1, keepdim=True)
              non_redundancy_interaction.append(dot)
      mixed_interaction = []
      for i in range(self.num_prefix):
          for j in range(self.num_sufix):
              v_ij = redundancy_ffm_embed[j - 1+self.num_prefix][ i, :]
              v_ji = non_redundancy_ffm_embed[i - 1][ :,j, :]
              dot = torch.sum(v_ij * v_ji, dim=-1, keepdim=True)
              mixed_interaction.append(dot)
      mixed = torch.concat(mixed_interaction,dim = -1)
      non_redundancy = torch.concat(non_redundancy_interaction,dim = -1)
      redundancy = torch.concat(redundancy_interaction,dim = -1).repeat(batch,1)
      return torch.concat([redundancy,non_redundancy,mixed],dim = -1)
  return pattern,ReplacementClass(),_match    

In [5]:
def workload_onn(num_field, prefix,dim = 64,l = [1024,512,256],batch = 4096):
  print(f"now gen workload of ONN with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  onn_model = onn.ONN([100 for i in range(num_field)],dim,l,0.1)

  model_traced_ori = symbolic_trace(onn_model)
  
  onn_model_modify = onn.ONN([100 for i in range(num_field)],dim,l,0.1)
  onn_model_traced_modify = symbolic_trace(onn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_ffm(onn_model_traced_modify,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=num_field,batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(onn_model_traced_modify, pattern, replace,[match])
  return model_traced_ori,onn_model_traced_modify

In [6]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [7]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_onn,l = [1024,512,256]):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
    return prof
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l,batch = batch)
  p1 = run(ori)
  p2 = run(modify)
  # return p1, p2

In [96]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :1024
now gen workload of ONN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1024 / 1024 (100.0%)
Greatest absolute difference: 0.08688631653785706 at index (278,) (up to 1e-05 allowed)
Greatest relative difference: 0.16523314632486433 at index (641,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:26:33 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:26:48 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:26:48 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(58.93664598464966, 106.18242251472884)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1023 / 1024 (99.9%)
Greatest absolute difference: 0.0837107002735138 at index (254,) (up to 1e-05 allowed)
Greatest relative difference: 0.18294408845936808 at index (751,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:27:59 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:28:03 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:28:03 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(29.351670742034912, 121.89810538990832)


In [97]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :2048
now gen workload of ONN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2048 / 2048 (100.0%)
Greatest absolute difference: 0.1027364432811737 at index (1894,) (up to 1e-05 allowed)
Greatest relative difference: 0.1882775361415543 at index (33,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:29:12 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:29:27 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:29:27 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(73.45104932785034, 98.58576149037481)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2046 / 2048 (99.9%)
Greatest absolute difference: 0.08499833941459656 at index (664,) (up to 1e-05 allowed)
Greatest relative difference: 0.16876641555310049 at index (1362,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:30:45 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:30:50 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:30:50 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(42.53497362136841, 126.36687574956204)


In [98]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :4096
now gen workload of ONN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4096 / 4096 (100.0%)
Greatest absolute difference: 0.08499535918235779 at index (1829,) (up to 1e-05 allowed)
Greatest relative difference: 0.16869988008247944 at index (1278,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:31:14 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:31:31 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:31:31 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(88.06931018829346, 103.04867842753447)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4094 / 4096 (100.0%)
Greatest absolute difference: 0.09317901730537415 at index (4006,) (up to 1e-05 allowed)
Greatest relative difference: 0.22173787452501872 at index (4006,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:32:50 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:32:56 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:32:56 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(51.175968647003174, 139.64064246039788)


In [99]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :8192
now gen workload of ONN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8187 / 8192 (99.9%)
Greatest absolute difference: 0.09401658177375793 at index (2077,) (up to 1e-05 allowed)
Greatest relative difference: 0.20544748201757446 at index (2077,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:33:19 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:33:39 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:33:39 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(118.1407356262207, 186.53886183383293)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8187 / 8192 (99.9%)
Greatest absolute difference: 0.09522327780723572 at index (724,) (up to 1e-05 allowed)
Greatest relative difference: 0.19618264624301768 at index (724,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:34:59 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:35:09 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:35:09 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(95.61225175857544, 185.15111005328336)


In [100]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :1024
now gen workload of ONN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1024 / 1024 (100.0%)
Greatest absolute difference: 0.09019425511360168 at index (996,) (up to 1e-05 allowed)
Greatest relative difference: 0.2181442146877612 at index (996,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:35:32 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:35:46 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:35:47 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(66.06680631637573, 104.86598084168554)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1023 / 1024 (99.9%)
Greatest absolute difference: 0.08098778128623962 at index (802,) (up to 1e-05 allowed)
Greatest relative difference: 0.19307929697927714 at index (506,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:37:09 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:37:13 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:37:13 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(37.70640850067139, 130.49009463204496)


In [101]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :2048
now gen workload of ONN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2046 / 2048 (99.9%)
Greatest absolute difference: 0.0830497145652771 at index (1510,) (up to 1e-05 allowed)
Greatest relative difference: 0.18317761694322623 at index (931,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:37:37 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:37:53 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:37:53 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(77.91058778762817, 111.9913351018281)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2047 / 2048 (100.0%)
Greatest absolute difference: 0.10086509585380554 at index (682,) (up to 1e-05 allowed)
Greatest relative difference: 0.23950308327921557 at index (682,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:39:33 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:39:39 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:39:39 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(53.25834512710571, 131.55890803170087)


In [102]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :4096
now gen workload of ONN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4095 / 4096 (100.0%)
Greatest absolute difference: 0.11890864372253418 at index (4088,) (up to 1e-05 allowed)
Greatest relative difference: 0.20943076512362305 at index (4088,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:40:08 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:40:28 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:40:28 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(112.42722749710083, 90.66726647512814)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4094 / 4096 (100.0%)
Greatest absolute difference: 0.1216464638710022 at index (2724,) (up to 1e-05 allowed)
Greatest relative difference: 0.31901741045064713 at index (2724,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:41:53 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:42:01 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:42:01 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(70.11226177215576, 143.07320449349754)


In [103]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :8192
now gen workload of ONN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8186 / 8192 (99.9%)
Greatest absolute difference: 0.10765805840492249 at index (2663,) (up to 1e-05 allowed)
Greatest relative difference: 0.23445804109682128 at index (2663,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:42:25 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:42:57 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:42:57 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(235.71604251861572, 137.43174714093038)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8184 / 8192 (99.9%)
Greatest absolute difference: 0.12393209338188171 at index (7003,) (up to 1e-05 allowed)
Greatest relative difference: 0.2531686482672423 at index (7003,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:44:21 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:44:31 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:44:31 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(92.42038011550903, 167.97751885412137)


In [104]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :1024
now gen workload of ONN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1023 / 1024 (99.9%)
Greatest absolute difference: 0.10393553972244263 at index (129,) (up to 1e-05 allowed)
Greatest relative difference: 0.23119723976523207 at index (129,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:45:00 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:46:02 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:46:02 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(183.2527494430542, 620.3820645983105)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1024 / 1024 (100.0%)
Greatest absolute difference: 0.0943833589553833 at index (418,) (up to 1e-05 allowed)
Greatest relative difference: 0.2100391522880352 at index (418,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 23:51:54 4187164:4187164 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 23:51:59 4187164:4187164 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 23:51:59 4187164:4187164 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(34.76699113845825, 749.8824384914038)


In [10]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :2048
now gen workload of ONN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2047 / 2048 (100.0%)
Greatest absolute difference: 0.10331553220748901 at index (224,) (up to 1e-05 allowed)
Greatest relative difference: 0.237432239884302 at index (224,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:04:53 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:06:10 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:06:10 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(215.9519362449646, 5868.606042402299)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2047 / 2048 (100.0%)
Greatest absolute difference: 0.0767166018486023 at index (1012,) (up to 1e-05 allowed)
Greatest relative difference: 0.1385295398492488 at index (794,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:11:33 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:11:39 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:11:39 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(39.83044385910034, 792.9121211685643)


In [11]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :4096
now gen workload of ONN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4094 / 4096 (100.0%)
Greatest absolute difference: 0.10926640033721924 at index (1202,) (up to 1e-05 allowed)
Greatest relative difference: 0.24908948212580448 at index (1202,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:12:37 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:13:50 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:13:51 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(245.86992263793945, 1157.0607363103118)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4091 / 4096 (99.9%)
Greatest absolute difference: 0.11779800057411194 at index (3367,) (up to 1e-05 allowed)
Greatest relative difference: 0.22071640666292333 at index (1373,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:20:09 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:20:15 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:20:15 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(53.41608285903931, 762.9208954646127)


In [12]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :8192
now gen workload of ONN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8188 / 8192 (100.0%)
Greatest absolute difference: 0.10884416103363037 at index (1764,) (up to 1e-05 allowed)
Greatest relative difference: 0.2562579879122093 at index (4341,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:21:14 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:22:40 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:22:40 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(403.6461138725281, 282.6970557424552)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8187 / 8192 (99.9%)
Greatest absolute difference: 0.09707492589950562 at index (3735,) (up to 1e-05 allowed)
Greatest relative difference: 0.22360326133367822 at index (3735,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:29:00 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:29:13 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:29:13 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(101.07780456542969, 859.7726376352948)


In [13]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :1024
now gen workload of ONN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1022 / 1024 (99.8%)
Greatest absolute difference: 0.09179344773292542 at index (269,) (up to 1e-05 allowed)
Greatest relative difference: 0.21445048721578444 at index (830,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:30:00 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:31:08 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:31:08 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(251.29464626312256, 450.7461920207097)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1022 / 1024 (99.8%)
Greatest absolute difference: 0.10423526167869568 at index (491,) (up to 1e-05 allowed)
Greatest relative difference: 0.23284802509699407 at index (491,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:37:35 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:37:41 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:37:41 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(43.87165546417236, 798.7143097566583)


In [14]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :2048
now gen workload of ONN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2047 / 2048 (100.0%)
Greatest absolute difference: 0.09592723846435547 at index (665,) (up to 1e-05 allowed)
Greatest relative difference: 0.19950029124644023 at index (198,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:38:25 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:39:35 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:39:35 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(270.95096349716187, 2243.3829888794835)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2048 / 2048 (100.0%)
Greatest absolute difference: 0.12129852175712585 at index (1518,) (up to 1e-05 allowed)
Greatest relative difference: 0.30030685862110107 at index (1518,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:45:52 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:46:00 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:46:00 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(62.43985652923584, 1128.2048872347127)


In [15]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :4096
now gen workload of ONN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4095 / 4096 (100.0%)
Greatest absolute difference: 0.11997002363204956 at index (150,) (up to 1e-05 allowed)
Greatest relative difference: 0.24415685703176043 at index (3266,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:46:57 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:48:18 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:48:18 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(374.88027811050415, 811.5965977098945)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4093 / 4096 (99.9%)
Greatest absolute difference: 0.12234768271446228 at index (1512,) (up to 1e-05 allowed)
Greatest relative difference: 0.25222226215698723 at index (1512,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:54:23 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:54:32 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:54:32 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(72.9086971282959, 787.8376593693226)


In [16]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :8192
now gen workload of ONN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8190 / 8192 (100.0%)
Greatest absolute difference: 0.10322993993759155 at index (7909,) (up to 1e-05 allowed)
Greatest relative difference: 0.2638639676012722 at index (268,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 02:55:29 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 02:57:14 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 02:57:15 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(615.9156608581543, 1390.824546633073)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8191 / 8192 (100.0%)
Greatest absolute difference: 0.1355355978012085 at index (941,) (up to 1e-05 allowed)
Greatest relative difference: 0.3009881987367889 at index (554,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-05 03:03:18 237897:237897 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-05 03:03:31 237897:237897 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-05 03:03:31 237897:237897 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(111.82626485824585, 780.6182877349959)


In [ ]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

In [ ]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

In [ ]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

In [ ]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

In [ ]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

In [ ]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

In [ ]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

In [ ]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

In [ ]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

In [ ]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

In [ ]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

In [8]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of ONN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4093 / 4096 (99.9%)
Greatest absolute difference: 0.15922138094902039 at index (44,) (up to 1e-05 allowed)
Greatest relative difference: 0.3588210382403885 at index (1471,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-07 21:38:40 2635888:2635888 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


: 